# Slippage Model

### Slippage is an important part of modelling performance. Without it, seemingly profitable strategies become unprofitable quickly -- or a favorite strategy can become nonviable based on a very limited capacity. 
#### One of my favorite instruments to trade is /RB -- Gasoline, a lightly traded Futures market, so it's a dangerous game of tracking capacity, and watching volume closely.  
To create an effective slippage model, there are a few factors (from incredibly simple), to not so simple, that can be accounted for.  In this example, I've included most everything from explicit slippage to implicit slippage -- but these models can be as simple as tracking 1-3% of ADV and ensuring your quantity is below that level.  
One variable I didn't include in calculations, but created support for, is volatility -- in incredibly volatile markets, orders that aren't filled immediately can move quickly against you or in favor of your order.  Keep in mind, when we're speaking of slippage -- moving AGAINST your trade is GOOD for slippage, as it gives you a more favorable average entry price. Movement in your trades direction (prior to order fills) is what we need to be careful of, and the more volatility there is -- the larger these moves can be, and the more detrimental to your avg fill price.


### Assignment 

- Create a slippage model that takes into account ADV and Volatility at a minimum (Sufficient default values are .025 of ADV).  For an extra challenge, take a shot at price impact, and other implicit slippage factors.  I've seen everything from ATR to ADV & Sigma used in these models, and complexity ranging from linear approaches to differential models, but keep in mind the most complex model is NOT always the best -- (I may have gone overboard).

- BONUS: Create an Object Oriented Approach, and see if you can integrate a Pandas DF as an instance variable for use with these calculations.  Keep in mind, this is likely a part of a larger system, with many moving parts. Always try to build reusable code segments that can be interchangeable with various systems.
    HINT: a Class could be useful, with instance vars and either class vars or globals.

- BONUS: See if you can simplify/speed up the necessary variables from DF into vectors


- BONUS: Create a testing tool to compute ranges of slippage based on these values

In [148]:
volume_share = .025
price_impact = .1

import math
import numpy as np
import statistics
import yfinance as yf
import pandas_datareader as pdr

class SlippageModel:
    volume_limit = .025
    price_impact = .1
    
    def __init__(self, sym, qty):
        self.symbol = sym
        self.qty = qty
        
        #Initialize a DF to use for other values!
        self.df = pdr.DataReader(sym,'yahoo',2019)
        self.volume = self.df.Volume.values #vol_array
        self.prices = self.df['Adj Close'].values #price_arr
        
        self.volume_share = self.qty / self.volume[-1] 
        
        self.price = self.prices[-1]
        self.min_vol = self.volume[-1]
        
        self.ADV = np.mean(self.volume[-20:])
        self.VOL = statistics.stdev(self.prices[-20:])  
        self.minute_data = []
        
    @classmethod
    def set_volume_limit(cls,vol_share):
        if vol_share > cls.volume_limit:
            print(f'WARNING -- raising Volume limit above default max value {cls.volume_share}')
        cls.volume_limit = vol_share
        return cls.volume_limit
    
    @classmethod
    def set_price_impact(cls,price_imp):
        if price_imp < cls.price_impact:
            print(f'WARNING -- lowering price_impact level below default value {cls.price_impact}')
        cls.price_impact = price_imp
        return cls.price_impact
    
    def replace_price_impact(self):
        '''Use sim to replace default price impact class var'''
        SlippageModel.price_impact = self.sim_impact
    
    
    def get_volume_share(self,qty=0, enforce_max=True,  volume_1m=None):
        '''Calculate + update volume share based on qty, and current volume'''
        if qty == 0: qty = self.qty
        if volume_1m is None: volume_1m = self.min_vol
        limit = self.volume_limit
        
        qty_of_barv = qty / volume_1m
        if qty_of_barv > limit: 
            print(f'WARNING -- Current QTY {qty} / {volume_1m} == {qty_of_barv} is > default max volume {limit}')
        if qty_of_barv > limit * 2:
            raise Exception(f'Qty too large -- MAX of .025, current at {qty_of_barv}')
            
        if enforce_max: 
            self.volume_share = min(qty_of_barv, limit)
        else:
            self.volume_share = qty_of_barv
        return self.volume_share
    
        
    def get_ADV(self): return self.ADV
    def get_VOL(self): return self.VOL
    
    
    def set_ADV(self,lookback=20):
        self.ADV = np.mean(self.volume[-lookback:]) #vol_array[-20:]
        return self.ADV
    
    def set_VOL(self,lookback=20):
        self.VOL = statistics.stdev(self.prices[-lookback:])
        return self.VOL
    
    def get_1m_vol(self):
        #Use YFinance! 
        data = yf.download(tickers=self.symbol,
        period="5d",
        interval="1m").drop(columns=['Open','High','Low','Close'])
        
        self.price = data['Adj Close'].iloc[-1] #Consider taking mean here instead? Median?
        
        self.minute_data = data.Volume.values
        self.minute_vol = self.minute_data[-1]
        
        return self.minute_vol
    
    def get_sim_impact(self,qty=0, price=None, eta=1.618/2, mean_volume=None, volatility=None):
        '''
        Use 20 day ADV for mean_volume, and 20d Sigma for Volatility
        CURRENTLY USING AN ETA THAT IS ENTIRELY ESTIMATED -- could look this up, or not sure...
        '''
        if qty == 0: qty = self.qty
        if price is None: price = self.price 
        if mean_volume is None: mean_volume = self.ADV
        if volatility is None: volatility = self.VOL

        #eta = 1.618 #CONSTANT TO LOOK UP (for now, golden ratio estimate)
        psi = qty / mean_volume

        market_impact = eta * volatility * math.sqrt(psi)
        #Returns PERCENT (I THINK?)
        self.sim_impact = market_impact #CONFIRM I DON't NEED TO DIVIDE THIS BY 100 ?
        
        #to get BASIS POINTS -- divide by 10000, PCT -- divide by 100 
        return (price * market_impact) / 100
    
    def test_slippage(self, qty=None, enforce_max=True, price_impact=None):
        '''Test various slippage levels at different quantities, with or without default MAX'''
        if price_impact is None: price_impact = self.price_impact
        if qty is None: 
            volume_share = self.volume_share 
            #qty = self.qty
        else:
            #if qty is None: qty = self.qty
            volume_share = self.get_volume_share(qty,enforce_max)
            #local_only volume_share option (to test other values)
            
        volume_pct = min(self.volume_limit,volume_share)
        return self.price * (1 + price_impact * (volume_pct ** 2))
            
        
    def get_real_slippage(self, price_impact=None, volume_share=None,side=1):
        '''Maybe this should SIMPLY include QTY? '''
        #if qty is None: qty = self.qty
        #if price is None: price = self.price
        if price_impact is None: price_impact = self.price_impact
        if volume_share is None: volume_share = self.volume_share
        volume_limit = self.volume_limit
        
        if self.volume_share > self.volume_limit:
            print('WARNING -- Qty is > Volume limit')
            
        
        volume_pct = min(self.volume_share,volume_limit)
        if side == 1:
            return self.price * (1 + price_impact * (volume_pct **2))
        return self.price * (1 - price_impact * (volume_pct ** 2))
    
    
    def calculate_slippage_range(self,pi_max,volume_min):
        '''Calculates a range of slippage based on price_impact and volume_min'''
        price = self.price
        if volume_min >= self.volume_limit:
            raise Exception(f'Set volume_min below {self.volume_limit}')
        if pi_max <= self.price_impact:
            raise Exception(f'Set pi_max above {self.price_impact}')
            

        slips = [self.get_real_slippage(p,v) for p in np.arange(self.price_impact, pi_max,.01) 
                 for v in np.arange(volume_min,self.volume_limit,.01)]
        
        self.slip_range = [ i - price for i in slips]
        self.slip_pcts = [(i-price)/price for i in slips]   #Looks wrong?
        return self.slip_range
                


    
    

    
    
    
    
    
    
        
    


t = SlippageModel('AAPL',10000)
t.set_volume_limit(.025)
t.set_price_impact(.1)
print(t.price_impact)
print(t.volume_limit)
t.get_1m_vol()
t.VOL

t.get_real_slippage()
t.test_slippage()


t.calculate_slippage_range(.2,.01)
t.get_sim_impact()


t.replace_price_impact()
t.price_impact
t.set_price_impact(.1)
t.calculate_slippage_range(.2,.01)

0.1
0.025
[*********************100%***********************]  1 of 1 completed
WARNING -- lowering price_impact level below default value 0.1981353847452624


[5.264769896484722e-05,
 5.264769896484722e-05,
 5.7912468889753654e-05,
 5.7912468889753654e-05,
 6.317723875781667e-05,
 6.317723875781667e-05,
 6.844200862587968e-05,
 6.844200862587968e-05,
 7.370677860762953e-05,
 7.370677860762953e-05,
 7.897154847569254e-05,
 7.897154847569254e-05,
 8.423631840059898e-05,
 8.423631840059898e-05,
 8.950108826866199e-05,
 8.950108826866199e-05,
 9.4765858136725e-05,
 9.4765858136725e-05,
 0.00010003062806163143,
 0.00010003062806163143]

1.013727529577479

0.1 0.01
0.1 0.02
0.11 0.01
0.11 0.02
0.12 0.01
0.12 0.02
0.13 0.01
0.13 0.02
0.13999999999999999 0.01
0.13999999999999999 0.02
0.14999999999999997 0.01
0.14999999999999997 0.02
0.15999999999999998 0.01
0.15999999999999998 0.02
0.16999999999999998 0.01
0.16999999999999998 0.02
0.17999999999999997 0.01
0.17999999999999997 0.02
0.18999999999999995 0.01
0.18999999999999995 0.02
0.19999999999999996 0.01
0.19999999999999996 0.02
0.20999999999999996 0.01
0.20999999999999996 0.02
0.21999999999999995 0.01
0.21999999999999995 0.02
0.22999999999999995 0.01
0.22999999999999995 0.02
0.23999999999999994 0.01
0.23999999999999994 0.02
